In [ ]:


p_ids = [
    "ESPORTSTMNT01:2730855",
    # "ESPORTSTMNT01:2730866",
    # "ESPORTSTMNT01:2731026",
]

In [22]:
import pandas as pd


df = pd.read_csv('sql/shutdown.csv')
df = df.sort_values(by=["platformgameid", "gametime"])
df.fillna(0, inplace=True)

new_row = {
    "platformgameid": None,
    "gametime": None,
    "shutdown_held_1": None,
    "shutdown_held_2": None,
    "shutdown_held_3": None,
    "shutdown_held_4": None,
    "shutdown_held_5": None,
    "shutdown_held_6": None,
    "shutdown_held_7": None,
    "shutdown_held_8": None,
    "shutdown_held_9": None,
    "shutdown_held_10": None,
}
shutdown_new = pd.read_csv('sql/shutdown.csv')
shutdown_new = shutdown_new.sort_values(by=["platformgameid", "gametime"])

new = []
prev_row = None
for _, row in shutdown_new.iterrows():
    if prev_row is None:
        prev_row = row
        new_row["platformgameid"] = row["platformgameid"]
        new_row["gametime"] = row["gametime"]
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
        continue
    elif row["platformgameid"] != prev_row["platformgameid"]:
        new.append(new_row)
        new_row = {
            "platformgameid": row["platformgameid"],
            "gametime": row["gametime"],
        }
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
    elif row["platformgameid"] == new_row["platformgameid"]:
        for i in range(1, 11):
            if row["shutdown_held_" + str(i)] > prev_row["shutdown_held_" + str(i)]:
                new_row["shutdown_held_" + str(i)] += row["shutdown_held_" + str(i)]- prev_row["shutdown_held_" + str(i)] 


    prev_row = row

shutdown = pd.DataFrame(new)


shutdown["blue_team_shutdown"] =  shutdown["shutdown_held_1"] + shutdown["shutdown_held_2"] + shutdown["shutdown_held_3"] + shutdown["shutdown_held_4"] + shutdown["shutdown_held_5"]
shutdown["red_team_shutdown"] = shutdown["shutdown_held_6"] + shutdown["shutdown_held_7"] + shutdown["shutdown_held_8"] + shutdown["shutdown_held_9"] + shutdown["shutdown_held_10"]
shutdown = shutdown.drop(columns=[
    "shutdown_held_1",
    "shutdown_held_2",
    "shutdown_held_3",
    "shutdown_held_4",
    "shutdown_held_5",
    "shutdown_held_6",
    "shutdown_held_7",
    "shutdown_held_8",
    "shutdown_held_9",
    "shutdown_held_10",
])
shutdown[shutdown["platformgameid"].isin(p_ids)].head(100)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown
1354,ESPORTSTMNT01:2730785,1059004,2900.0,1150.0


In [25]:
p_ids = [
    "ESPORTSTMNT01:2730785",
    "ESPORTSTMNT01:2730866",
    "ESPORTSTMNT01:2731026",
]

shutdown[shutdown["platformgameid"].isin(p_ids)].head(100)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown
1354,ESPORTSTMNT01:2730785,1059004,2900.0,1150.0
1357,ESPORTSTMNT01:2730866,1379018,1350.0,0.0
1358,ESPORTSTMNT01:2731026,341021,1000.0,1400.0


In [32]:
shutdown_collected = pd.read_csv('sql/shutdown_collected.csv')
shutdown_collected.fillna(0, inplace=True)
shutdown_collected.sort_values(by=["platformgameid"], inplace=True)

blue_side = shutdown_collected[shutdown_collected["side"] == 100]
blue_side = blue_side.rename(columns={
    "shutdownbounty": "blue_shutdown_collected",
})
blue_side.drop(columns=["side", "teamid"], inplace=True)
red_side = shutdown_collected[shutdown_collected["side"] == 200]
red_side = red_side.rename(columns={
    "shutdownbounty": "red_shutdown_collected",
})
red_side.drop(columns=[
    "side",
    "teamid",
], inplace=True)

shutdown_collected = blue_side.merge(red_side, on="platformgameid")
display(shutdown_collected[shutdown_collected["platformgameid"].isin(p_ids)].head(100))

,platformgameid,blue_shutdown_collected,red_shutdown_collected
2414,ESPORTSTMNT01:2730785,600.0,300.0
2417,ESPORTSTMNT01:2730866,150.0,850.0
2418,ESPORTSTMNT01:2731026,1550.0,400.0
